# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# PIPELINE EXTRAÇÃO

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
import pandas as pd
import yt_dlp
import os

path_sql = "repository\\querys\\operation\\insert\\video_historic.sql"
with open(path_sql, "r", encoding="utf-8") as file:
    sql_query_insert = file.read()
   
with open("repository\\querys\\operation\\read\\video.sql", "r", encoding="utf-8") as file:
    sql_query_read = file.read()
    df_videos = db_manager.read_df(sql_query_read)

In [ ]:
for ID_MOVIE, ID_VIDEO, URL in zip(df_videos["id_movie"], df_videos["id_video"], df_videos["url"]):
    for video_info in video.get_video_info(url_or_id=URL):
        break
        # cut = df_videos[df_videos.id_video.isin([ID_VIDEO])]
        # if cut.empty:
        URL             = video["url"]
        TITLE           = video["title"]
        ID_PLAYLIST     = ID_PLAYLIST
        ID_GAME         = ID_GAME
        ID_PROPLAYER    = None
        ID_HERO         = None
        TIME_MOVIE      = video["duration_seconds"] if "duration_seconds" in video else -1
        PUBLISHED_DATE  = video["published_date"] if "published_date" in video else "" 
        
        sql_template            = Template(sql_query_insert)
        rendered_sql            = sql_template.render(
            ID_MOVIE            = ID_MOVIE,
            TITLE               = TITLE,
            DESCRIPTION         = video["description"] if "description" in video else "",
            VIEWS               = video["views"] if "views" in video else 0,
            LIKES               = video["likes"] if "likes" in video else 0,
            LIVE                = video["live"] if "live" in video else False,
            KEYWORDS            = video["keywords"] if "keywords" in video else "",
            AVAILABLE_COUNTRIES = video["available_countries"] if "available_countries" in video else "",
            CATEGORY            = video["category"] if "category" in video else "",
            FAMILY_FRIENDLY     = video["family_friendly"] if "family_friendly" in video else False,
            DT_UPLOAD           = PUBLISHED_DATE,
            DT_PUBLISH          = PUBLISHED_DATE,
            DT_START            = datetime.now().strftime("%Y-%m-%d"),
            DT_END              = None
        ).replace("'None'", "NULL").replace("'False'", "FALSE").replace("'True'", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
            print(f"✓ Video historic inserido: {TITLE}")
        except Exception as e:
            print(f"Error inserting video data: {e}")
            print(f"SQL: {rendered_sql}")
            db_manager.connection.rollback()
    break
